# 使用FP-growth高效发现频繁项集

FP（Frequent Pattern）频繁模式，一种存储项集出现次数的数据结构

growth —— 生长，一种可以让树自增高的算法
* Apriori算法产生候选项集，扫描数据集来检查他们是否频繁
* FP-growth算法改进Apriori，将数据集存储在FP树中，通过查找元素项的条件基以及构建条件FP树来发现频繁项集。

## 构建FP树的数据结构
* FP树的结点定义

In [3]:
class tree_node:
    def __init__(self, name, num, parent):
        self.name = name
        self.num = num
        self.node_link = None
        self.parent = parent
        self.child = {}

* 节点数增加

In [1]:
def inc(self, num):
    self.num += num

* 遍历节点

In [12]:
def disp(self, ind=1):
    print(' ' * ind,self.name, '\t', self.num)
    for child in self.child.values():
        disp(child, ind + 1)

In [15]:
root = tree_node('python', 8, None)
child1 = tree_node('java', 1, root)
root.child['java'] = child1
child2 = tree_node('c++', 5, root)
root.child['c++'] = child2

In [16]:
disp(root)

  python 	 8
   java 	 1
   c++ 	 5


## 构建PF树

* 对数据集进行两次遍历

第一次遍历：

    计算每个元素项的频率
    去掉不满足最小支持度的项，排序
第二次遍历：

    自上而下遍历树节点
    如果当前元素节点存在，则增加此节点的值
    如果当前元素节点不存在，则添加一个分支节点

生长函数，元素项存在则计数，不存在则生长

In [17]:
def update_tree(item, tree, header_table, count):
    if item[0] in tree.child:
        tree.child[item[0]].inc(count)
    else:
        tree.child[item[0]] = tree(item[0], count, tree)
        if header_table[item[0]][1] == None:
            header_table[item[0]][1] = tree.child[item[0]]
        else:
            update_header(header_table[item[0][1]], tree.child[item[0]])
    if len(item) > 1:
        update_tree(item[1::], tree.child[item[0]], header_table, count)

In [18]:
def update_header(node_to_test, target_node):
    while node_to_test.node_link != None:
        node_to_test = node_to_test.node_link
    node_to_test.node_link = target_node

In [19]:
def create_tree(dataset, min_sup=1):
    header_table = {}
    for trans in dataset:
        for item in trans:
            header_table[item] = header_table.get(item, 0) + dataset[trans]
    for k in header_table.keys():
        if header_table[k] < min_sup:
            del(header_table[k])
    freq_item_set = set(header_table.keys())
    if len(freq_item_set) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]
    ret_tree = tree_node('Null Set', 1, None)
    for tran_set, count in dataset.items():
        localD = {}
        for item in tran_set:
            if item in freq_item_set:
                localD[item] = header_table[item][0]
        if len(localD) > 0:
            ordered_item = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            update_tree(ordered_item, ret_tree, header_table, count)
    return ret_tree, header_table